# 决策树简介
决策树通过定义一系列的**判断模块**（ decision block ），来决定数据的最终标签（分类问题）。

原理： 从训练集数据中抽取出一系列的判断模块，形成决策树，测试数据通过经过判断模块来决定最终落入哪个分类。
适用数据类型： 数值型 和 标称型；

优点： 
- 数据形式易于理解，决策依据易于理解，保留数据的内在含义（这一点 KNN 做不到）；
- 计算复杂度不高；
- 对中间值的缺失不敏感；
- 可以处理不相关特征数据；

缺点：
- 容易产生过拟合；

基本实现过程：
1. 定义数据特征；
2. 利用信息熵的计算公式，对比所有数据特征作为判断模块所使用的数据特征时产生的信息增益，使用信息增益最高的特征作为判断模块的指标；
3. 根据当前判断模块所使用的指标的所有值（去重）分别建立分支，并遍历分支：
    - 如果分支中的实例都是相同标签，则生成一个终止模块（ terminating block );
    - 如果分支中的实例存在不同的标签：
        - 如果还有未使用的特征，则重新进行步骤 2；
        - 如果没有未使用的特征，则使用实例中数量最多的标签建立终止模块；
4. 最终形成的决策树的所有叶子节点均为标签终止模块，决策树模型建立完成；
5. 测试数据从决策树的根节点开始进行判断，最终到达一个终止模块，决定其标签；


信息熵：度量信源的平均不确定性；


In [1]:
def create_data():
    # 生成数据
    training_data = [
                        [1,1,"yes"],
                        [1,1,"yes"],
                        [1,0,"no"],
                        [0,1,"no"],
                        [0,1,"no"],
                    ]
    
    feature_name = ["no surfing", "flippers"]
    return training_data, feature_name

training_data, feature_names = create_data()
print(training_data)

# {'flippers': {0: 'no', 1: {'no surfing': {0: 'no', 1: 'yes'}}}}



[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]


In [2]:
from math import log

def calculate_shannon_ent(data_set):
    # 计算给定数据集的香农熵 
    # data_set 为数组形式的数组， data_set 的每一个元素为一个实例，且实例最后一个维度为标签
    ret = 0.0

    row_count = len(data_set)    
    label_val_set = set([item[-1] for item in data_set])
    label_val_count_dict = dict()
    
    for row_data in data_set:
        label_val = row_data[-1]
        if label_val not in label_val_count_dict:
            label_val_count_dict[label_val] = 0
        label_val_count_dict[label_val] += 1
        
    for label_val, label_count in label_val_count_dict.items():
        possiblity = float(label_count) / float(row_count)
        ret -= possiblity * log(possiblity, 2)
        
    return ret
    
# calculate the original shannon_ent for training data
print(calculate_shannon_ent(training_data))

0.9709505944546686


In [4]:
def split_data_set(data_set, feature_index, feature_val):
    # 筛选出对应特征为指定特征值的记录并移除对应的特征
    ret = list()
    
    for item in data_set:
        if item[feature_index] != feature_val:
            continue
        else:
            sub_item_left = item[:feature_index]
            sub_item_right = item[feature_index+1:]
            
            sub_item_left.extend(sub_item_right)
            ret.append(sub_item_left)
    return ret

# print(training_data)
# print(split_data_set(training_data,1,1))
            


def choose_cur_best_feature(data_set, feature_names):
    # 对比当前各个特征作为决策模块后的信息熵，取最高的特征之一作为当前最佳特征
    feature_shannon_ent_dict = dict()    
    feature_count = len(feature_names)
    max_shannon_ent = 0.0
    max_shannon_feature_index = 0

    best_feature = -1
    
    for i in range(feature_count):
        feature_name = feature_names[i]
        cur_feature_shannon_ent = 0.0
        feature_val_set = set([ item[i] for item in data_set])
        for feature_val in feature_val_set:
            sub_data_set = split_data_set(data_set, i, feature_val)
            sub_prob = len(sub_data_set)/len(data_set)
            sub_shannon_ent = calculate_shannon_ent(sub_data_set)
            cur_feature_shannon_ent += sub_prob * sub_shannon_ent
        if cur_feature_shannon_ent >= max_shannon_ent:
            max_shannon_ent = cur_feature_shannon_ent
            best_feature = i
        print("The shannon_ent for feature[%s] is [%s]..." % (feature_names[i], sub_shannon_ent))
    print("The best feature for current is %s" % feature_names[best_feature])
    return best_feature, feature_names[best_feature]


choose_cur_best_feature(training_data, feature_names)

The shannon_ent for feature[no surfing] is [0.9182958340544896]...
The shannon_ent for feature[flippers] is [1.0]...
The best feature for current is flippers


(1, 'flippers')

In [5]:
feature_name_key = "feature_name"
decisions_key = "decisions"
result_key = "result"

import operator
import json
def get_majority(data_set):
    """
        获取数据集中比重最大的标签
    """
    label_count_dict = dict()
    
    for item in data_set:
        label = item[-1]
        if label not in label_count_dict:
            label_count_dict[label] = 0
        label_count_dict += 1
    
    sorted_labels = sorted(data_set.iteritems(), key = operator.itemgetter(1), reverse= True)
    return sorted_labels[0][0]
    
def build_decision_tree(data_set, feature_names):
    """
        构建决策树
        {'flippers': {0: 'no', 1: {'no surfing': {0: 'no', 1: 'yes'}}}}
    

    - 如果当前数据中所有实例的标签相同，直接返回标签；
    - 否则：        
        - 选择最佳特征，根据最佳特征的取值对数据进行分组；
            - 遍历每个分组
                - 分组数据内没有特征（当前特征是当前分支唯一特征），则取数据中占大多数的标签作为结果；

                
    """    
    label_list = [item[-1] for item in data_set]
    if len(set(label_list)) == 1:
        # stop splitting when all of the labels are equal.
        print("All labels are equal...")
        return label_list[0]
    elif len(feature_names) == 0:
        # stop splitting when there is no more feature.
        print("No more feature...")
        return get_majority(data_set)
        pass
    else:
        cur_feature_id, cur_feature_name = choose_cur_best_feature(data_set, feature_names)
        
        cur_feature_vals = [ item[cur_feature_id] for item in data_set]
        cur_feature_vals = set(cur_feature_vals)
        
        ret = {
            cur_feature_name : dict()
        }
        
        del feature_names[cur_feature_id]
        
        for feature_val in cur_feature_vals:
            sub_labels = feature_names[:]
            ret[cur_feature_name][feature_val] = build_decision_tree(split_data_set(data_set,cur_feature_id,feature_val), sub_labels)

        return ret
    

print(build_decision_tree(training_data, feature_names))

The shannon_ent for feature[no surfing] is [0.9182958340544896]...
The shannon_ent for feature[flippers] is [1.0]...
The best feature for current is flippers
All labels are equal...
The shannon_ent for feature[no surfing] is [0.0]...
The best feature for current is no surfing
All labels are equal...
All labels are equal...
{'flippers': {0: 'no', 1: {'no surfing': {0: 'no', 1: 'yes'}}}}


In [13]:
file_name = "lenses.txt"
file_handler = open(file_name)

data = file_handler.readlines()
row_count = len(data)

data_set = list()
labels = list()

for item in data:
    item = item[:-2]
    data_set.append(item.split("\t"))

for i in range(len(data_set[0])-1):
    labels.append("f-%s" % i)

print(json.dumps({"data" : data_set}, ensure_ascii = False))
print(labels)

print(build_decision_tree(data_set, labels))

{"data": [["young", "myope", "no", "reduced", "no lense"], ["young", "myope", "no", "normal", "sof"], ["young", "myope", "yes", "reduced", "no lense"], ["young", "myope", "yes", "normal", "har"], ["young", "hyper", "no", "reduced", "no lense"], ["young", "hyper", "no", "normal", "sof"], ["young", "hyper", "yes", "reduced", "no lense"], ["young", "hyper", "yes", "normal", "har"], ["pre", "myope", "no", "reduced", "no lense"], ["pre", "myope", "no", "normal", "sof"], ["pre", "myope", "yes", "reduced", "no lense"], ["pre", "myope", "yes", "normal", "har"], ["pre", "hyper", "no", "reduced", "no lense"], ["pre", "hyper", "no", "normal", "sof"], ["pre", "hyper", "yes", "reduced", "no lense"], ["pre", "hyper", "yes", "normal", "no lense"], ["presbyopic", "myope", "no", "reduced", "no lense"], ["presbyopic", "myope", "no", "normal", "no lense"], ["presbyopic", "myope", "yes", "reduced", "no lense"], ["presbyopic", "myope", "yes", "normal", "har"], ["presbyopic", "hyper", "no", "reduced", "no l